In [ ]:
access_token = ""

In [2]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, default_data_collator, EarlyStoppingCallback, IntervalStrategy
#from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score # Import all needed metrics

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')

Device: cuda:0


In [4]:
dataset = pd.read_csv('/home/patrick.araujo/llama2/datasets/balanced_output_train.csv')

In [5]:
dataset

,level_0,index,Unnamed: 0,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,sentiment,lengthContent,Language
0,1007,1527,1527,7d6a4b7f-0f17-47c6-8010-3187dd1c86a7,I've been using my visa gift card. All the inf...,1,2,26.23.4.100,2023-12-28 21:20:56,NaN,NaN,26.23.4.100,0,224,en
1,8622,12941,12941,227c23e5-e178-4d69-bb99-3fe1445dc035,the PRIME PRICE IS WAY TO HIGH!!,5,0,26.19.2.100,2023-10-02 12:09:02,NaN,NaN,26.19.2.100,2,32,en
2,7774,11420,11420,bc748a49-bbd9-4773-a63f-f0950ad66310,easy to use and fast free delivery,5,0,26.20.0.100,2023-10-25 03:46:34,NaN,NaN,26.20.0.100,2,34,en
3,5478,8135,8135,0b2c1d90-3026-427a-bea9-918d1e067e8f,The Shopping On Amazon Is The Greatest Of All....,5,0,NaN,2023-11-12 01:54:20,NaN,NaN,NaN,2,111,en
4,13393,19072,19072,bee9613a-352d-4328-abaa-986316fe788d,I am rarely not satisfied seems I'm always sat...,4,0,24.22.0.100,2023-01-10 09:30:07,NaN,NaN,24.22.0.100,2,64,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9307,7444,10822,10822,e95c926d-6510-4b97-8ad7-7b11885a46a7,Amazon is always my go to for everything!!,5,0,26.21.0.100,2023-11-03 01:06:14,NaN,NaN,26.21.0.100,2,42,en
9308,17040,24320,24320,f26c2ae2-6868-49ab-9845-963ce24f14f8,This is a great app but I didn't get my neckla...,3,1,22.14.0.100,2022-06-01 13:35:57,NaN,NaN,22.14.0.100,1,240,en
9309,14542,20557,20557,cd102f54-3164-494c-8287-b2064c82401b,11-3-22- I have 107 S&S items. It is impossibl...,4,9,24.20.2.100,2022-11-03 11:59:23,NaN,NaN,24.20.2.100,2,320,en
9310,16186,22964,22964,d8372268-d77e-4666-a2dd-166decb97443,"App sucks! Glitchy, slow, hurts my eyes cuz im...",3,0,24.12.6.100,2022-07-21 14:35:03,NaN,NaN,24.12.6.100,1,231,en


In [6]:
dataset.shape

(9312, 15)

In [7]:
sentiment_counts_balanced = dataset['sentiment'].value_counts().sort_index()
print(sentiment_counts_balanced)

sentiment
0    3104
1    3104
2    3104
Name: count, dtype: int64


In [8]:
# Load the pre-trained BERT model and tokenizer
#model_name = "/home/patrick.araujo/llama2/llama/llama-2-7b-hf"
#model_name = "mistralai/Mistral-7B-v0.1"
model_name = "bert-base-uncased"
#model_name = "google-t5/t5-base"
#model_name = "google-t5/t5-small"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3, token=access_token)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# Create train and validation datasets
comments = dataset[['content', 'sentiment']]
train_data, val_data = train_test_split(comments, test_size=0.2, random_state=42)

In [10]:
train_data

,content,sentiment
503,Usually love the app but lately if I try to us...,1
5813,It's okay sometimes if you really follow the s...,2
274,Here's to all the people that make all the goo...,2
1345,It's amazing but some of the stuff I want isn'...,2
5057,How disgraceful all these complaints and still...,0
...,...,...
5734,No longer works with Tablets mine is a Samsung...,0
5191,Just wish we had more American Made products t...,2
5390,Excellent customer service. So many choices! B...,2
860,"Amazon prices aren't bad, they deliver fairly ...",2


In [11]:
val_data

,content,sentiment
8638,Amazon shopping app doesn't allow user to clea...,1
131,Can't vote my orders. It just refreshes my ord...,1
3509,"App is barely better than web experience, but ...",2
4401,Wishlist scrolling doesn't work,0
3095,With the problems of item deliveries thru USPS...,0
...,...,...
6657,Have to watch an add before you can use the ap...,0
4271,"App is not bad, but I used it mostly to track ...",1
2404,I dispise that the app has injected a search i...,0
6603,I've officially deleted my account after using...,0


In [12]:
#tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_size = 'right'

In [13]:
max_length = 512

def tokenizerFunction(string):
  # Tokenize a list of texts and return a dictionary with the tokenization outputs
  tokens = tokenizer(string, padding='max_length', truncation=True, max_length=max_length, return_tensors="pt")
  return tokens

train_encodings = train_data['content'].apply(tokenizerFunction)
val_encodings = val_data['content'].apply(tokenizerFunction)

In [14]:
train_encodings

503     [input_ids, token_type_ids, attention_mask]
5813    [input_ids, token_type_ids, attention_mask]
274     [input_ids, token_type_ids, attention_mask]
1345    [input_ids, token_type_ids, attention_mask]
5057    [input_ids, token_type_ids, attention_mask]
                           ...                     
5734    [input_ids, token_type_ids, attention_mask]
5191    [input_ids, token_type_ids, attention_mask]
5390    [input_ids, token_type_ids, attention_mask]
860     [input_ids, token_type_ids, attention_mask]
7270    [input_ids, token_type_ids, attention_mask]
Name: content, Length: 7449, dtype: object

In [15]:
val_encodings

8638    [input_ids, token_type_ids, attention_mask]
131     [input_ids, token_type_ids, attention_mask]
3509    [input_ids, token_type_ids, attention_mask]
4401    [input_ids, token_type_ids, attention_mask]
3095    [input_ids, token_type_ids, attention_mask]
                           ...                     
6657    [input_ids, token_type_ids, attention_mask]
4271    [input_ids, token_type_ids, attention_mask]
2404    [input_ids, token_type_ids, attention_mask]
6603    [input_ids, token_type_ids, attention_mask]
2362    [input_ids, token_type_ids, attention_mask]
Name: content, Length: 1863, dtype: object

In [16]:
def creatingDataframe(encodings):
  indices = encodings.index.to_numpy()

  df = pd.DataFrame(columns=['input_ids', 'attention_mask'])
  
  # Assuming 'content' is a Series or List containing tuples/lists
  for element in encodings:
    input_ids = element['input_ids'][0].numpy()              # Replace with correct index based on your data structure
    attention_mask = element['attention_mask'][0].numpy()    # Replace with correct index
    df.loc[len(df.index)] = [input_ids, attention_mask]
  df_index = pd.DataFrame(indices, columns=['index'])
  df['index'] = df_index['index']
  return df

In [17]:
# Create datasets
# Convert labels to tensors
train_dataset = creatingDataframe(train_encodings)
train_labels = train_data['sentiment'].reset_index(name='label')
train_dataset = pd.merge(train_dataset, train_labels, on='index')

val_dataset = creatingDataframe(val_encodings)
val_labels = val_data['sentiment'].reset_index(name='label')
val_dataset = pd.merge(val_dataset, val_labels, on='index')

# Define batch size and create data loaders
batch_size = 8  # Adjust as needed
# train_dataset = SentimentDataset(train_encodings, train_labels)
# val_dataset = SentimentDataset(val_encodings, val_labels)
#train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=default_data_collator)
#val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=default_data_collator)

In [18]:
train_dataset = train_dataset.drop('index', axis=1)
val_dataset = val_dataset.drop('index', axis=1)

In [19]:
train_dataset

,input_ids,attention_mask,label
0,"[101, 2788, 2293, 1996, 10439, 2021, 9906, 206...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
1,"[101, 2009, 1005, 1055, 3100, 2823, 2065, 2017...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...",2
2,"[101, 2182, 1005, 1055, 2000, 2035, 1996, 2111...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
3,"[101, 2009, 1005, 1055, 6429, 2021, 2070, 1997...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
4,"[101, 2129, 29591, 3993, 2035, 2122, 10821, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
...,...,...,...
7444,"[101, 2053, 2936, 2573, 2007, 17596, 3067, 200...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
7445,"[101, 2074, 4299, 2057, 2018, 2062, 2137, 2081...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...",2
7446,"[101, 6581, 8013, 2326, 1012, 2061, 2116, 9804...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
7447,"[101, 9733, 7597, 4995, 1005, 1056, 2919, 1010...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2


In [20]:
val_dataset

,input_ids,attention_mask,label
0,"[101, 9733, 6023, 10439, 2987, 1005, 1056, 349...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
1,"[101, 2064, 1005, 1056, 3789, 2026, 4449, 1012...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
2,"[101, 10439, 2003, 4510, 2488, 2084, 4773, 332...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
3,"[101, 4299, 9863, 28903, 2987, 1005, 1056, 214...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...",0
4,"[101, 2007, 1996, 3471, 1997, 8875, 23534, 270...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
...,...,...,...
1858,"[101, 2031, 2000, 3422, 2019, 5587, 2077, 2017...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
1859,"[101, 10439, 2003, 2025, 2919, 1010, 2021, 104...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
1860,"[101, 1045, 4487, 13102, 5562, 2008, 1996, 104...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
1861,"[101, 1045, 1005, 2310, 3985, 17159, 2026, 407...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0


In [21]:
# Define a custom dataset class
#class CustomDataset(Dataset):
#    def __init__(self, dataframe, data):
#        self._data = data
#
#    def __len__(self):
#        return len(self._data)
#
#    def __getitem__(self, idx):
#        label = self.labels[idx]
#        input_ids = self.input_ids[idx]
#        attention_mask = self.attention_masks[idx]
#        return {
#            'input_ids': torch.tensor(dataframe['input_ids'], dtype=torch.long),
#            'attention_mask': torch.tensor(dataframe['attention_mask'], dtype=torch.long),
#            'labels': torch.tensor(dataframe['label'], dtype=torch.long)
#        }

In [22]:
#train_dataset = CustomDataset(train_dataset, train_data)
#val_dataset = CustomDataset(val_dataset, val_data)

train_dataset = Dataset.from_pandas(train_dataset)
val_dataset = Dataset.from_pandas(val_dataset)

In [23]:
# Define training arguments with early stopping
training_args = TrainingArguments(
    # Positional arguments:
    output_dir="./bert-base-uncased_sentiment_model",
    logging_dir="./logs_b",

    # Keyword arguments:
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    #eval_steps=250,                         # Adjust as needed
    #save_total_limit=2,                     # Adjust as needed. Only last 2 models are saved. Older ones are deleted.
    num_train_epochs=10,                     # Adjust as needed
    #save_steps=500,                         # Adjust as needed
    metric_for_best_model="eval_loss",      # Use validation loss for early stopping
    save_strategy="epoch",
    load_best_model_at_end=True
)

In [25]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred

    # Convert logits to predicted labels
    predicted_labels = np.argmax(logits, axis=1)

    metrics = {}
    # Calculate accuracy
    accuracy = accuracy_score(labels, predicted_labels)
    metrics['accuracy'] = accuracy

    # Calculate precision, recall, F1 score, and support for each class
    precision, recall, f1, support = precision_recall_fscore_support(labels, predicted_labels)    
    

    # Create a dictionary to store metrics for each class
    class_metrics = {}
    for i in range(len(precision)):
        class_metrics[f'class_{i}'] = {
            'precision': precision[i],
            'recall': recall[i],
            'f1': f1[i],
            'support': support[i]
        }

    precision_ALL = precision_score(labels, predicted_labels, average='macro')
    recall_ALL = recall_score(labels, predicted_labels, average='macro')
    f1_ALL = f1_score(labels, predicted_labels, average='macro')

    metrics['precision'] = precision_ALL
    metrics['recall'] = recall_ALL
    metrics['f1'] = f1_ALL
    
    # Print and return the metrics
    for class_name, c_metrics in class_metrics.items():
        metrics[f'{class_name}_precision'] = c_metrics["precision"]
        metrics[f'{class_name}_recall'] = c_metrics["recall"]
        metrics[f'{class_name}_f1'] = c_metrics["f1"]
        metrics[f'{class_name}_support'] = c_metrics["support"]

    return metrics

In [26]:
# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=default_data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [27]:
# Fine-tune the model
trainer.train()

/home/patrick.araujo/miniconda3/envs/llama/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Class 0 Precision,Class 0 Recall,Class 0 F1,Class 0 Support,Class 1 Precision,Class 1 Recall,Class 1 F1,Class 1 Support,Class 2 Precision,Class 2 Recall,Class 2 F1,Class 2 Support
1,No log,0.769734,0.630166,0.674967,0.631928,0.637272,0.713974,0.512539,0.596715,638,0.472928,0.707438,0.566887,605,0.838000,0.675806,0.748214,620
2,No log,0.779572,0.618357,0.680006,0.620441,0.626735,0.684322,0.506270,0.581982,638,0.465768,0.742149,0.572339,605,0.889930,0.612903,0.725883,620
3,No log,0.903259,0.634461,0.664288,0.633635,0.641116,0.633968,0.678683,0.655564,638,0.486933,0.585124,0.531532,605,0.871965,0.637097,0.736253,620
4,0.638900,1.180173,0.623725,0.659980,0.624551,0.632003,0.667286,0.562696,0.610544,638,0.469679,0.652893,0.546335,605,0.842975,0.658065,0.739130,620


Checkpoint destination directory ./bert-base-uncased_sentiment_model/checkpoint-156 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/home/patrick.araujo/miniconda3/envs/llama/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Checkpoint destination directory ./bert-base-uncased_sentiment_model/checkpoint-312 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/home/patrick.araujo/miniconda3/envs/llama/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Checkpoint destination directory ./bert-base-unc

TrainOutput(global_step=624, training_loss=0.5663309708619729, metrics={'train_runtime': 564.0739, 'train_samples_per_second': 330.143, 'train_steps_per_second': 6.914, 'total_flos': 7839727394623488.0, 'train_loss': 0.5663309708619729, 'epoch': 4.0})

In [28]:
# Evaluate the model
results = trainer.evaluate()

/home/patrick.araujo/miniconda3/envs/llama/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [29]:
results

{'eval_loss': 0.7697339653968811,
 'eval_accuracy': 0.6301663982823403,
 'eval_precision': 0.6749673253074059,
 'eval_recall': 0.6319278843649356,
 'eval_f1': 0.637272343799994,
 'eval_class_0_precision': 0.7139737991266376,
 'eval_class_0_recall': 0.512539184952978,
 'eval_class_0_f1': 0.5967153284671532,
 'eval_class_0_support': 638,
 'eval_class_1_precision': 0.47292817679558014,
 'eval_class_1_recall': 0.7074380165289256,
 'eval_class_1_f1': 0.5668874172185431,
 'eval_class_1_support': 605,
 'eval_class_2_precision': 0.838,
 'eval_class_2_recall': 0.6758064516129032,
 'eval_class_2_f1': 0.7482142857142857,
 'eval_class_2_support': 620,
 'eval_runtime': 11.6259,
 'eval_samples_per_second': 160.245,
 'eval_steps_per_second': 3.355,
 'epoch': 4.0}

In [30]:
# Save the model
model.save_pretrained("./bert-base-uncased_sentiment_model")